# Packages

In [ ]:
import pandas as pd
import os
import sklearn
import matplotlib.pyplot as plt
import numpy as np

# Encodage et séparation des jeux de données
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

# Classifieur bayésien
from sklearn.naive_bayes import GaussianNB

# MLP
from sklearn.neural_network import MLPClassifier

# Arbre
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report

# Random forest
from sklearn.ensemble import RandomForestClassifier

# Précision du modèle
from sklearn.metrics import confusion_matrix, accuracy_score

import warnings
warnings.filterwarnings('ignore')

# Import des données

In [ ]:
ot_odr_filename = os.path.join(".", "OT_ODR.csv.bz2")
ot_odr_df = pd.read_csv(ot_odr_filename,
                        compression="bz2",
                        sep=";")

equipements_filename = os.path.join(".", 'EQUIPEMENTS.csv')
equipements_df = pd.read_csv(equipements_filename,
                             sep=";")

# Nettoyage et traitement des données

In [ ]:
# Jointure entre les deux jeus de données
data = pd.merge(ot_odr_df, equipements_df, on='EQU_ID')

# Définir les intervalles de 50 000 km pour la colonne kilométrage
bins = range(0, int(data["KILOMETRAGE"].max()) + 50000, 50000)
labels = [f'{i}-{i+49999}' for i in bins[:-1]]

# Créer une nouvelle colonne avec les classes de 50 000 km
data['KILOMETRAGE_CLASSE'] = pd.cut(data["KILOMETRAGE"], bins=bins, labels=labels, include_lowest=True)

# dumie de la variable sig_contexte
dummies = data['SIG_CONTEXTE'].str.get_dummies(sep='/')
data_origine = pd.concat([data, dummies], axis=1)

# Prédiction SYSTEM_N1, SYSTEM_N2 et SYSTEM_N3 

<div style="text-align: center; color: blue;">
    <h1><em>Classifieur bayésien naïf</em></h1>
</div>

In [ ]:
def train_and_evaluate_model(data, target_variable):
    # Liste des colonnes à supprimer
    columns_to_drop = ["DUREE_TRAVAIL", "TYPE_TRAVAIL", "SIG_CONTEXTE", "ODR_LIBELLE", "ODR_ID", "OT_ID", 
                       "EQU_ID", "KILOMETRAGE", "DATE_OT"]
    columns_to_drop += [col for col in ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3"] if col != target_variable]

    # Préparation des données
    data_target = data.drop(columns_to_drop, axis=1)
    
    # Séparer les variables explicatives (X) et la variable à prédire (y)
    X = data_target.drop([target_variable], axis=1)
    y = data_target.filter([target_variable])
    
    # Encodage des variables utiles
    le = LabelEncoder()
    for column in X.columns:
        X[column] = le.fit_transform(X[column])

    # Séparer les données en ensembles d'entraînement et de test
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=5)

    # Entraînement du modèle
    naive_bayes = GaussianNB()
    naive_bayes.fit(x_train, y_train.values.ravel())

    # Prédictions
    y_pred = naive_bayes.predict(x_test)

    # Évaluation du modèle
    accuracy = accuracy_score(y_test, y_pred)
    matrice_confusion = confusion_matrix(y_test, y_pred)

    print(f'\nModèle pour {target_variable}:')
    print(f'Accuracy : {accuracy}')
    print('Confusion Matrix :')
    print(matrice_confusion)
    
    return accuracy, matrice_confusion

# Exécution du modèle pour SYSTEM_N1, SYSTEM_N2, et SYSTEM_N3
for target in ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3"]:
    accuracy, matrice_confusion = train_and_evaluate_model(data, target)

<div style="text-align: center; color: blue;">
    <h1><em>MLP</em></h1>
</div>

In [ ]:
def train_and_evaluate_model(data, target_variable):
    # Liste des colonnes à supprimer
    columns_to_drop = ["DUREE_TRAVAIL", "TYPE_TRAVAIL", "SIG_CONTEXTE", "ODR_LIBELLE", "ODR_ID", "OT_ID", 
                       "EQU_ID", "KILOMETRAGE", "DATE_OT"]
    columns_to_drop += [col for col in ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3"] if col != target_variable]

    # Préparation des données
    data_target = data.drop(columns_to_drop, axis=1)
    
    # Séparer les variables explicatives (X) et la variable à prédire (y)
    X = data_target.drop([target_variable], axis=1)
    y = data_target.filter([target_variable])
    
    # Encodage des variables utiles
    le = LabelEncoder()
    for column in X.columns:
        X[column] = le.fit_transform(X[column])

    # Séparer les données en ensembles d'entraînement et de test
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=5)

    # MLP avec 1 couche cachée et 10 neurones, fonction d'activation RELU
    clf = MLPClassifier(random_state=1,hidden_layer_sizes=(20,), activation='relu',max_iter=20).fit(x_train, y_train)
    y_pred = clf.predict(x_test)

    # Évaluation du modèle
    accuracy = accuracy_score(y_test, y_pred)
    matrice_confusion = confusion_matrix(y_test, y_pred)

    print(f'\nModèle pour {target_variable}:')
    print(f'Accuracy : {accuracy}')
    print('Confusion Matrix :')
    print(matrice_confusion)
    
    return accuracy, matrice_confusion

# Exécution du modèle pour SYSTEM_N1, SYSTEM_N2, et SYSTEM_N3
for target in ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3"]:
    accuracy, matrice_confusion = train_and_evaluate_model(data, target)

<div style="text-align: center; color: blue;">
    <h1><em>Arbre de décision</em></h1>
</div>

In [ ]:
def train_and_evaluate_model_tree(data, target_variable, A_pred):
    # Liste des colonnes à supprimer
    columns_to_drop = ["DUREE_TRAVAIL", "TYPE_TRAVAIL", "SIG_CONTEXTE", "ODR_ID", "OT_ID", 
                       "EQU_ID", "KILOMETRAGE", "DATE_OT"]
    columns_to_drop += [col for col in ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3","ODR_LIBELLE"] if col != target_variable]

    # Préparation des données
    data_target = data.drop(columns_to_drop, axis=1)

    # Séparer les variables explicatives (X) et la variable à prédire (y)
    X = data_target.drop([target_variable], axis=1)
    y = data_target[target_variable]

    # Séparer les données en ensembles d'entraînement et de test
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=5)

    # Encodage des variables utiles sur l'ensemble d'entraînement
    label_encoders = {}
    for column in x_train.columns:
        le = LabelEncoder()
        x_train[column] = le.fit_transform(x_train[column])
        # Encoder les valeurs de x_test qui sont aussi présentes dans x_train
        x_test[column] = x_test[column].map(lambda s: '<unknown>' if s not in le.classes_ else s)
        le.classes_ = np.append(le.classes_, '<unknown>')
        x_test[column] = le.transform(x_test[column])
        label_encoders[column] = le

    # Encodage de la variable cible sur l'ensemble d'entraînement
    le_target = LabelEncoder()
    y_train = le_target.fit_transform(y_train)
    y_test = y_test.map(lambda s: '<unknown>' if s not in le_target.classes_ else s)
    le_target.classes_ = np.append(le_target.classes_, '<unknown>')
    y_test = le_target.transform(y_test)

    # Entraînement du modèle
    tree = DecisionTreeClassifier(max_depth=14)
    tree.fit(x_train, y_train)

    # Prédictions
    y_pred = tree.predict(x_test)
    y_pred_proba = tree.predict_proba(x_test)

    # Évaluation du modèle
    accuracy = accuracy_score(y_test, y_pred)
    matrice_confusion = confusion_matrix(y_test, y_pred)

    # Prédiction avec de nouvelles données (A_pred)
    A_pred = pd.DataFrame([A_pred], columns=x_train.columns)

    for column in A_pred.columns:
        if column in label_encoders:
            le = label_encoders[column]
            A_pred[column] = A_pred[column].map(lambda s: '<unknown>' if s not in le.classes_ else s)
            A_pred[column] = le.transform(A_pred[column])
    
    pred_proba = tree.predict_proba(A_pred)[0]  # Probabilités pour la première ligne de A_pred

    # Obtenir la classe prédite avec la probabilité maximale
    max_proba_index = np.argmax(pred_proba)
    max_proba_class = le_target.inverse_transform([max_proba_index])[0]
    max_proba_value = np.max(pred_proba)

    return accuracy, max_proba_class, max_proba_value

# Exécution du modèle pour SYSTEM_N1, SYSTEM_N2, SYSTEM_N3, et ODR_LIBELLE avec A_pred en entrée
A_pred = ["GLACE/BAIE", "DEBOITE", "L0482", "MD017", "C007", "MT014", "100000-149999"]

for target in ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3", "ODR_LIBELLE"]:
    accuracy, max_proba_class, max_proba_value = train_and_evaluate_model_tree(data, target, A_pred)
    print(f'\nModèle pour {target}:')
    print(f'Accuracy : {accuracy}')
    #print('Confusion Matrix :')
    #print(matrice_confusion)
    print(f'Classe prédite avec la probabilité la plus élevée : {max_proba_class}')
    print(f'Probabilité : {max_proba_value}')

<div style="text-align: center; color: blue;">
    <h1><em>Random Forest</em></h1>
</div>

In [ ]:
def train_and_evaluate_model_random_forest(data, target_variable):
    # Liste des colonnes à supprimer
    columns_to_drop = ["DUREE_TRAVAIL", "TYPE_TRAVAIL", "SIG_CONTEXTE", "ODR_LIBELLE", "ODR_ID", "OT_ID", 
                       "EQU_ID", "KILOMETRAGE", "DATE_OT"]
    columns_to_drop += [col for col in ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3"] if col != target_variable]

    # Préparation des données
    data_target = data.drop(columns_to_drop, axis=1)

    # Encodage des variables utiles
    le = LabelEncoder()
    for column in data_target.columns:
        data_target[column] = le.fit_transform(data_target[column])
    
    # Séparer les variables explicatives (X) et la variable à prédire (y)
    X = data_target.drop([target_variable], axis=1)
    y = data_target.filter([target_variable])

    # Séparer les données en ensembles d'entraînement et de test
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=5)

    # Entraînement du modèle
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=20)
    rf_model.fit(x_train, y_train)

    # Prédictions
    y_pred = rf_model.predict(x_test)

    # Évaluation du modèle
    accuracy = accuracy_score(y_test, y_pred)
    matrice_confusion = confusion_matrix(y_test, y_pred)

    print(f'\nModèle pour {target_variable}:')
    print(f'Accuracy : {accuracy}')
    print('Confusion Matrix :')
    print(matrice_confusion)

    return accuracy, matrice_confusion

# Exécution du modèle pour SYSTEM_N1, SYSTEM_N2, et SYSTEM_N3
for target in ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3"]:
    accuracy, matrice_confusion = train_and_evaluate_model_random_forest(data, target)


<div style="text-align: center; color: blue;">
    <h1><em>GridSearchCV</em></h1>
</div>

In [ ]:
def train_and_evaluate_model(data, target_variable):
    # Liste des colonnes à supprimer
    columns_to_drop = ["DUREE_TRAVAIL", "TYPE_TRAVAIL", "SIG_CONTEXTE", "ODR_LIBELLE", "ODR_ID", "OT_ID", 
                       "EQU_ID", "KILOMETRAGE", "DATE_OT"] 
    columns_to_drop += [col for col in ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3"] if col != target_variable]

    # Préparation des données
    data_target = data.drop(columns_to_drop, axis=1)
    
    # Séparer les variables explicatives (X) et la variable à prédire (y)
    X = data_target.drop([target_variable], axis=1)
    y = data_target[target_variable]
    
    # Encodage des variables utiles
    le = LabelEncoder()
    for column in X.columns:
        X[column] = le.fit_transform(X[column])

    # Séparer les données en ensembles d'entraînement et de test
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=5)

    # Définir le modèle et les paramètres pour GridSearchCV
    model = GaussianNB()
    param_grid = {
        'var_smoothing': np.logspace(0, -9, num=100)  # Hyperparamètre à optimiser
    }

    # Utiliser GridSearchCV pour trouver les meilleurs hyperparamètres
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(x_train, y_train.values.ravel())

    # Meilleur modèle et hyperparamètres
    best_model = grid_search.best_estimator_
    print(f'\nMeilleurs paramètres pour {target_variable}: {grid_search.best_params_}')

    # Prédictions
    y_pred = best_model.predict(x_test)
    y_pred_proba = best_model.predict_proba(x_test)

    # Évaluation du modèle
    accuracy = accuracy_score(y_test, y_pred)
    matrice_confusion = confusion_matrix(y_test, y_pred)

    print(f'\nModèle pour {target_variable}:')
    print(f'Accuracy : {accuracy}')
    print('Confusion Matrix :')
    print(matrice_confusion)
    
    return accuracy, matrice_confusion

# Exécution du modèle pour SYSTEM_N1, SYSTEM_N2, et SYSTEM_N3
for target in ["SYSTEM_N1", "SYSTEM_N2", "SYSTEM_N3"]:
    accuracy, matrice_confusion = train_and_evaluate_model(data, target)

# Prédiction ODR_LIBELLE

<div style="text-align: center; color: red;">
    <h1><em>Préparation</em></h1>
</div>

In [ ]:
data_odr = data.drop(["DUREE_TRAVAIL", "TYPE_TRAVAIL", "SIG_CONTEXTE", "SYSTEM_N3", "ODR_ID", "OT_ID", 
                     "EQU_ID", "SYSTEM_N1", "SYSTEM_N2", "KILOMETRAGE", "DATE_OT"], axis=1)


# Encodage des variables utiles
le = LabelEncoder()
for column in data_odr.columns:
    data_odr[column] = le.fit_transform(data_odr[column])

# Séparer les variables explicatives (X) et la variable à prédire (y)
X = data_odr.drop(["ODR_LIBELLE"],axis=1)
y = data_odr.filter(["ODR_LIBELLE"])

# Séparer les données en ensembles d'entraînement et de test
x_test, x_train, y_test, y_train = train_test_split(X, y, test_size=0.4, random_state = 5)

<div style="text-align: center; color: blue;">
    <h1><em>Classifieur bayésien naïf</em></h1>
</div>

In [ ]:
naive_bayes = GaussianNB() 
naive_bayes.fit(x_train, y_train)
y_pred = naive_bayes.predict(x_test)
# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy : {accuracy}')

In [ ]:
matrice_confusion = confusion_matrix(y_test, y_pred)
print(matrice_confusion)

<div style="text-align: center; color: blue;">
    <h1><em>MLP</em></h1>
</div>

In [ ]:
# MLP avec 1 couche cachée et 10 neurones, fonction d'activation RELU
clf = MLPClassifier(random_state=1,hidden_layer_sizes=(20,), activation='relu',max_iter=20).fit(x_train, y_train)
y_pred = clf.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy : {accuracy}')

In [ ]:
matrice_confusion = confusion_matrix(y_test, y_pred)
print(matrice_confusion)

In [ ]:
# MLP avec 1 couche cachée et 10 neurones, fonction d'activation Sigmoide
clf = MLPClassifier(random_state=1,hidden_layer_sizes=(25,), activation='tanh',solver='sgd',
                   learning_rate = 'adaptive', ,max_iter=20).fit(x_train, y_train)
y_pred = clf.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy : {accuracy}')

In [ ]:
matrice_confusion = confusion_matrix(y_test, y_pred)
print(matrice_confusion)

<div style="text-align: center; color: blue;">
    <h1><em>Arbre de décision</em></h1>
</div>

In [ ]:
for i in range (1,20):

    # Construire et entraîner l'arbre de décision
    tree1 = DecisionTreeClassifier(max_depth=i)
    tree1.fit(x_train, y_train)

    # Représentation de l'arbre (texte)
    text_representation = tree.export_text(tree1, feature_names=list(x_train.columns))
    #print(text_representation)

    # Représentation de l'arbre (graphique)
    #fig = plt.figure(figsize=(20, 10))
    #tree_plot = tree.plot_tree(tree1, feature_names=X_train.columns, class_names=[str(cls) for cls in tree1.classes_], filled=True)
    #plt.show()

    # Prédiction et matrice de confusion
    pred = tree1.predict(x_test)
    conf_matrix = confusion_matrix(y_test, pred)
    #print("Matrice de confusion:\n", conf_matrix)

    # Calcul de l'accuracy
    accuracy = accuracy_score(y_test, pred)
    print(i,"Précision (Accuracy):", accuracy)

    # Génération du rapport de classification
    class_report = classification_report(y_test, pred)
    #print("Rapport de classification:\n", class_report)

<div style="text-align: center; color: blue;">
    <h1><em>Random Forest</em></h1>
</div>

In [ ]:
for i in range (2,20):
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=i)

    # Entraînement du modèle
    rf_model.fit(x_train, y_train)

    # Prédiction sur l'ensemble de test
    y_pred = rf_model.predict(x_test)

    # Évaluation de la précision du modèle
    accuracy = accuracy_score(y_test, y_pred)
    print(i, f"Précision du modèle: {accuracy:.2f}")